In [1]:
# Look The Note Below!!!

import numpy as np
import time

def my_range(start, end):
    if start <= end:
        return range(start, end + 1)
    else:
        return range(start, end - 1, -1)


class Problem:
    char_mapping = ('·', 'Q')

    def __init__(self, n=4):
        self.n = n

    def is_valid(self, state):
        """
        check the state satisfy condition or not.
        :param state: list of points (in (row id, col id) tuple form)
        :return: bool value of valid or not
        """
        board = self.get_board(state)
        res = True
        for point in state:
            i, j = point
            condition1 = board[:, j].sum() <= 1
            condition2 = board[i, :].sum() <= 1
            condition3 = self.pos_slant_condition(board, point)
            condition4 = self.neg_slant_condition(board, point)
            res = res and condition1 and condition2 and condition3 and condition4
            if not res:
                break
        return res

    def is_satisfy(self, state):
        return self.is_valid(state) and len(state) == self.n

    def pos_slant_condition(self, board, point):
        i, j = point
        tmp = min(self.n - i - 1, j)
        start = (i + tmp, j - tmp)
        tmp = min(i, self.n - j - 1)
        end = (i - tmp,  j + tmp)
        rows = my_range(start[0], end[0])
        cols = my_range(start[1], end[1])
        return board[rows, cols].sum() <= 1

    def neg_slant_condition(self, board, point):
        i, j = point
        tmp = min(i, j)
        start = (i - tmp, j - tmp)
        tmp = min(self.n - i - 1, self.n - j - 1)
        end = (i + tmp, j + tmp)
        rows = my_range(start[0], end[0])
        cols = my_range(start[1], end[1])
        return board[rows, cols].sum() <= 1

    def get_board(self, state):
        board = np.zeros([self.n, self.n], dtype=int)
        for point in state:
            board[point] = 1
        return board

    def print_state(self, state):
        board = self.get_board(state)
        print('_' * (2 * self.n + 1))
        for row in board:
            for item in row:
                print(f'|{Problem.char_mapping[item]}', end='')
            print('|')
        print('-' * (2 * self.n + 1))

# Read the note First ! ! !
#### In all the code above, only two of the function usages are needed. One is is_valid(self, state), which is to determine if the current state is legal; the other is is_satisfy(self, state), which is to determine if the current board meets the win condition. 
#### The type of state is [], which stores the tuples(a, b), representing the positions of the queens in it.
#### In the first line of the code you can see N = 5, this is the size of the test.
#### Then in the test_block that follows, render indicates whether to use a graphical interface representation, and then method indicates which bts are used(bts or improving_bts).

### Request 1: You should complete the function bts(problem). 
You can only use iterative way, not recursive. Using recursion will incur a **20% penalty**. And you can add any function you want. (**DDL: 3.31**)
### Request 2: You should complete the function improving_bts(problem). 
You can select one or more methods of the three methods below(Minimum Remaining Value, Least constraining value, Forward checking), but you should have a good performance **when N = 16 without GUI**. (**DDL: 4.07**)

# BTS: Backtracking search (Request 1, DDL: 3.31)

In [2]:
def bts(problem):
    action_stack = []
    action_backtrace = {}
    board = {}
    directions = [(1, 1), (1, -1), (-1, 1), (-1, -1), (1, 0), (-1, 0), (0, 1), (0, -1)]
    pos_list = set([(i, j) for i in range(problem.n) for j in range(problem.n)])

    while not problem.is_satisfy(action_stack):
        # Place a Queen on the board

        forbidden = set([value for lst in board.values() for value in lst])
        forbidden.update(set([value for lst in action_backtrace.values() for value in lst]))
        avail_pos = list(pos_list - forbidden)
        # if there is not available slot, go back
        if not avail_pos:
            # print(f'size_of_stack={len(action_stack)}, content={action_stack}\n\t size_of_board={len(board.values())}, board={board}')
            if action_stack:
                last_pos = action_stack.pop()
                del board[last_pos]
                action_backtrace.pop(last_pos, None)
            continue
        new_pos = avail_pos[0]

        # then there is one. Add constraints.
        if action_stack:
            if not action_backtrace.get(action_stack[-1]):
                action_backtrace[action_stack[-1]] = []
            action_backtrace[action_stack[-1]].append(new_pos)
        action_stack.append(new_pos)

        # propagating constraints
        if not board.get(new_pos):
            board[new_pos] = [new_pos]
        for dx, dy in directions:
            i0 = new_pos[0] + dx
            j0 = new_pos[1] + dy
            while i0 < problem.n and i0 >= 0 and j0 < problem.n and j0 >= 0:
                board[new_pos].append((i0, j0))
                i0 += dx
                j0 += dy
            
        yield action_stack
        # You can know what yield means in CSDN~
        

# Improving BTS To DO (Request 2, DDL: 4.07)
* Which variable should be assigned next?
* In what order should its values be tried?
* Can we detect inevitable failure early?

### Minimum Remaining Value
Choose the variable with the fewest legal values in its domain
### Least constraining value
Given a variable, choose the least constraining value: the one that rules out the fewest values in the remaining variables
### Forward checking
Keep track of remaining legal values for the unassigned variables. Terminate when any variable has no legal values.

In [3]:
directions = [(1, 1), (1, -1), (-1, 1), (-1, -1),
              (1, 0), (-1, 0), (0, 1), (0, -1)]

# directions = [(1, 1), (1, -1), (1, 0), (0, 1)]
def compute_new_board0(ref_lst, new_pos, n):
    for dx, dy in directions:
        i0 = new_pos[0] + dx
        j0 = new_pos[1] + dy
        while i0 < n and i0 >= 0 and j0 < n and j0 >= 0:
            ref_lst.append((i0, j0))
            i0 += dx
            j0 += dy

def improved_bts0(problem):
    n = problem.n

    action_stack = []
    action_backtrace = {}
    board = {}
    pos_set = set([(i, j) for i in range(n)
                   for j in range(n)])
    # loop_range = np.array([i for i in range(-problem.n + 1, 0)] + [i for i in range(1, problem.n)])
    # print(f'{loop_range}')

    while not problem.is_satisfy(action_stack):
        # Place a Queen on the board

        forbidden = set([value for lst in board.values() for value in lst])
        forbidden.update(
            set([value for lst in action_backtrace.values() for value in lst]))
        avail_pos = list(pos_set - forbidden)

        # if there is not available slot, go back
        if not avail_pos:
            # print(f'size_of_stack={len(action_stack)}, content={action_stack}\n\t size_of_board={len(board.values())}, board={board}')
            if action_stack:
                last_pos = action_stack.pop()
                del board[last_pos]
                action_backtrace.pop(last_pos, None)
            continue

        # pick the next position
        new_pos = avail_pos[0]

        # forward checking
        if action_stack:
            if not action_backtrace.get(action_stack[-1]):
                action_backtrace[action_stack[-1]] = []
            action_backtrace[action_stack[-1]].append(new_pos)
        action_stack.append(new_pos)

        # appending constraints
        if not board.get(new_pos):
            board[new_pos] = [new_pos]
        ref_lst = board[new_pos]
        compute_new_board0(ref_lst, new_pos, n)

        # for dx, dy in directions:
        #     dxs = loop_range * dx + new_pos[0]
        #     dys = loop_range * dy + new_pos[1]
        #     ref_lst.extend(list(zip(dxs, dys)))

        yield action_stack
        # You can know what yield means in CSDN~


In [4]:
def compute_new_board1(ref_board, new_pos, n):
    res = ref_board.copy()
    res[new_pos] = 1
    for dx, dy in directions:
        i0 = new_pos[0] + dx
        j0 = new_pos[1] + dy
        while i0 < n and i0 >= 0 and j0 < n and j0 >= 0:
            res[i0, j0] = 1
            i0 += dx
            j0 += dy
    return res


def improved_bts1(problem: Problem):
    n = problem.n
    n_squared = n * n
    ref_board = np.zeros((n, n))

    action_stack = [(-1, -1)]
    forbidden_stack = [np.copy(ref_board).reshape((n, n))]
    assigned_stack = [-1 for i in range(n)]

    # Algorithm
    # Each time, assign row positions.
    # Select the row with fewest available slots

    while not problem.is_satisfy(action_stack[1:]):
        pending = [i for i in range(n) if assigned_stack[i] == -1]
        candidates = []

        last_board = forbidden_stack[-1]
        avail_slots = [(i, np.count_nonzero(last_board[i]))
                       for i in pending]
        
        # forward checking
        # this predicate check prunes the branching significantly
        if not any([x[1] == n for x in avail_slots]):
            candidates = [i for i in map(
                lambda x: x[0],
                sorted(
                    filter(lambda x: x[1] < n, avail_slots),
                    key=lambda x: x[1],
                    reverse=True))]

        # go back if no candidate
        if not candidates:
            if len(action_stack) > 1:
                last_pos = action_stack.pop()
                del forbidden_stack[-1]  # delete its forbidden stack
                assigned_stack[last_pos[0]] = -1  # unassign the row variable
                continue
            else:
                break

        # minimum remaining value (row)
        row = candidates[0]

        # least constraining value
        candidate_cols = []
        for col in range(n):
            if forbidden_stack[-1][row, col] == 0:  # if can be assigned
                # print(f'candidate ({row}, {col}) picked.')
                board = compute_new_board1(forbidden_stack[-1], (row, col), n)
                candidate_cols.append((col, board, np.count_nonzero(board)))

        candidate_cols.sort(key=lambda x: x[2])

        # pick the least constraining one
        # print(f'candidate_cols = {candidate_cols}')
        # print(f'action_stack = {action_stack}')
        col = candidate_cols[0][0]
        new_pos = (row, col)
        # print(f'examing position: ({row}, {col})', end='\r')

        action_stack.append(new_pos)
        forbidden_stack[-1][(new_pos)] = 1
        forbidden_stack.append(candidate_cols[0][1])
        assigned_stack[row] = col

        yield action_stack[1:]
        # You can know what yield means in CSDN~


In [10]:

# test_block
def test(n, method):
    # n = 8 # Do not modify this parameter, if you want to change the size, go to the first line of whole program.
    render = False # here to select GUI or not
    p = Problem(n)
    if render:
        import pygame
        w, h = 90 * n + 10, 90 * n + 10
        screen = pygame.display.set_mode((w, h))
        screen.fill('white')
        action_generator = method(p)
        clk = pygame.time.Clock()
        queen_img = pygame.image.load('./queen.png')
        while True:
            for event in pygame.event.get():
                if event == pygame.QUIT:
                    exit()
            try:
                actions = next(action_generator)
                screen.fill('white')
                for i in range(n + 1):
                    pygame.draw.rect(screen, 'black', (i * 90, 0, 10, h))
                    pygame.draw.rect(screen, 'black', (0, i * 90, w, 10))
                for action in actions:
                    i, j = action
                    screen.blit(queen_img, (10 + 90 * j, 10 + 90 * i))
                pygame.display.flip()
            except StopIteration:
                pass
            clk.tick(5)
        pass
    else:
        start_time = time.time()
        for actions in method(p):
            pass
        p.print_state(actions)
        print(time.time() - start_time)


import cProfile
cProfile.run('test(16, improved_bts1)')
# test(8, improved_bts1)
# test()

_________________________________
|Q|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|
|·|·|·|Q|·|·|·|·|·|·|·|·|·|·|·|·|
|·|·|·|·|·|·|Q|·|·|·|·|·|·|·|·|·|
|·|·|·|·|·|·|·|·|·|Q|·|·|·|·|·|·|
|·|·|·|·|·|·|·|·|·|·|·|Q|·|·|·|·|
|·|·|·|·|·|·|·|·|Q|·|·|·|·|·|·|·|
|·|Q|·|·|·|·|·|·|·|·|·|·|·|·|·|·|
|·|·|·|·|·|·|·|·|·|·|·|·|·|·|·|Q|
|·|·|Q|·|·|·|·|·|·|·|·|·|·|·|·|·|
|·|·|·|·|·|·|·|·|·|·|·|·|·|·|Q|·|
|·|·|·|·|·|·|·|Q|·|·|·|·|·|·|·|·|
|·|·|·|·|·|·|·|·|·|·|Q|·|·|·|·|·|
|·|·|·|·|·|·|·|·|·|·|·|·|·|Q|·|·|
|·|·|·|·|·|Q|·|·|·|·|·|·|·|·|·|·|
|·|·|·|·|·|·|·|·|·|·|·|·|Q|·|·|·|
|·|·|·|·|Q|·|·|·|·|·|·|·|·|·|·|·|
---------------------------------
0.4020388126373291
         195150 function calls in 0.403 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.000    0.000 2524431920.py:16(__init__)
      649    0.024    0.000    0.356    0.001 2524431920.py:19(is_valid)
      649    0.001    0.000    0.357    0.001 2524431920.py:38(is_satisf